In [115]:
import numpy as np

import torch
import torch.nn as nn

from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score

import matplotlib.pyplot as plt

gpu = torch.cuda.current_device()
torch.cuda.get_device_name()

'NVIDIA GeForce GTX 1650 with Max-Q Design'

In [116]:
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

n_samples, n_features = X.shape

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# use StandardScalar
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# convert all to torch tensor
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
# y_test = torch.from_numpy(y_test.astype(np.float32))

# now y is row vector with 1 row
# convert it to column vector
y_train = y_train.view(y_train.shape[0], 1)
# y_test = y_test.view(y_test.shape[0], 1)

`y_train[:5]` ->

without `.view()`:

`tensor([1., 0., 1., 1., 1.])`

with `.view()`:

`tensor([[1.],
        [0.],
        [1.],
        [1.],
        [1.]])`

In [117]:
# model

class LogisticRegression(nn.Module):

    def __init__(self, in_feat):
        super(LogisticRegression, self).__init__()

        # define layers
        self.linear = nn.Linear(in_feat, 1)

    def forward(self, X):
        y_predicted = self.linear(X)
        torch.sigmoid_(y_predicted)
        return y_predicted

model = LogisticRegression(n_features)

In [118]:
# loss(criterion) and optimizer
learning_rate = 0.001

criterion = nn.BCELoss()
optimizer = torch.optim.SGD(params=model.parameters(), lr=learning_rate)

In [119]:
# forward pass
num_epochs = 10000

for epoch in range(num_epochs):

    # forward pass
    y_predicted = model(X_train)

    # loss
    l = criterion(y_predicted, y_train)

    # backward pass
    l.backward()

    # optimizer step
    optimizer.step()
    optimizer.zero_grad()

    # print info
    if epoch % 1000 == 0:
        [w, b] = model.parameters()
        print(f'epoch {epoch+1}: w = {w[0][0].item():.3f}, loss = {l:.8f}')

epoch 1: w = -0.070, loss = 0.76638734
epoch 1001: w = -0.238, loss = 0.24936886
epoch 2001: w = -0.302, loss = 0.18529230
epoch 3001: w = -0.343, loss = 0.15676373
epoch 4001: w = -0.373, loss = 0.13982613
epoch 5001: w = -0.397, loss = 0.12833521
epoch 6001: w = -0.417, loss = 0.11990382
epoch 7001: w = -0.433, loss = 0.11338872
epoch 8001: w = -0.448, loss = 0.10816608
epoch 9001: w = -0.461, loss = 0.10386313


In [120]:
# do testing

with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_class = y_predicted.round().numpy()

    roc = roc_auc_score(y_test, y_predicted)
    print(f"roc: {roc:.4f}")


roc: 0.9950
